**Задача 1**
Создайте датафрейм содержащий сумму и количество продаж треков в разбивке по странам и континентам.
Создайте scatter plot показывающий соотношение суммы продаж и количества треков.
Добавьте разделение по цвету на основе континента. Цвет Азии должен быть жёлтым, цвет Европы синим, цвет Северной Америки красным.
Добавьте название диаграмме. Шрифт должен быть 30, цвет шрифта серый.
Измените шрифт тиков (значения осей) на 14, цвет на серый.

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import numpy as np
import pandas as pd
import plotly.express as px

from connect import set_connection

query = """
select
    i.billing_country,
    case 
        when i.billing_country in ('USA', 'Canada', 'Mexico') then 'North America'
        when i.billing_country in ('Germany', 'France', 'Italy', 'Spain', 'Netherlands', 'United Kingdom') then 'Europe'
        when i.billing_country in ('China', 'India', 'Japan', 'South Korea', 'Australia') then 'Asia'
        else 'Other'
    end as continent,
    sum(il.quantity * il.unit_price) as total_sales,
    sum(il.quantity) as total_quantity
from 
    invoice_line il
join 
    invoice i on il.invoice_id = i.invoice_id
group by 
    i.billing_country, continent
order by
    continent;
"""


with set_connection() as conn:
    df = pd.read_sql(query, conn)

fig = px.scatter(df, 
                 x='total_quantity', 
                 y='total_sales', 
                 color='continent', 
                 color_discrete_map={'North America': 'red', 'Europe': 'blue', 'Asia': 'yellow'},
                 title='Соотношение суммы продаж и количества треков',
                 labels ={'total_quantity':'tracks','total_sales':'USD'})

fig.update_layout(
    title_font=dict(size=30, color='gray'),
    xaxis_tickfont=dict(size=14, color='gray'),
    yaxis_tickfont=dict(size=14, color='gray')
)

fig.show()


**Задача 2**
Создайте датафрейм содержащий количество треков в разбивке по названиям жанров.
Создайте столбчатую диаграмму. По оси X должны быть названия жанров, по оси Y - количество треков.
Кастомизируйте диаграмму так, чтобы столбики всех жанров были серого цвета. За исключением жанров Jazz, Rock и Alternative & Punk. Эти жанры должны выделяться цветом. Цвета можете выбирать сами.
Уберите легенду.
Добавьте название диаграмме. Название должно быть курсивным.
Фон диаграммы должен быть белым.

In [3]:

query = """
select 
    g.name as genre_name,
    count(t.track_id) as track_count
from 
    genre g
left join 
    track t on g.genre_id = t.genre_id
group by
    g.name
order by 
    track_count desc;
"""
with set_connection() as conn:
    df = pd.read_sql(query, conn)

highlight_genres = ['Jazz', 'Rock', 'Alternative & Punk']
df['color'] = df['genre_name'].apply(lambda x: 'gray' if x not in highlight_genres else x)

fig = px.bar(df, 
             x='genre_name', 
             y='track_count', 
             color='color', 
             color_discrete_map={'Jazz': 'blue', 'Rock': 'green', 'Alternative & Punk': 'purple', 'gray': 'gray'},
             title='Количество треков по жанрам')

fig.update_layout(showlegend=False)
fig.update_layout(
    title_font=dict(size=30, color='black', family='Arial'),
    plot_bgcolor='white', 
    xaxis_title='',  
    yaxis_title='', 
    xaxis_tickangle=45,  
    xaxis_tickfont=dict(size=14, color='black'),
    yaxis_tickfont=dict(size=14, color='black')
)

fig.layout.title.text = "<i>Количество треков по жанрам</i>"

fig.show()
